In [240]:
from google.colab import drive
drive.mount('/content/drive')
ruta = "/content/drive/MyDrive/Colab Notebooks/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [241]:
! pip install pyspark

In [242]:
#Crear sesion Spark
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark = SparkSession.builder.appName("SparkSQLEjercicio2").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)

In [243]:
#Llamada a los archivos
from pyspark.sql.functions import to_date
dfp1 = spark.read.format("csv").option("mode", "FAILFAST")\
  .option("header", "true")\
  .option("inferSchema", "true")\
  .load(ruta+"/data/Litros_y_Servicios_ASA_2015-2022_modificado.csv")

dfe = spark.read.format("json").option("mode", "FAILFAST")\
  .option("inferSchema", "true")\
  .load(ruta+"/data/entidades_federativas_mexico.json")

In [244]:
#Dataframe "Servicios"
print(dfp.show())
print(dfp.count())

+-----------+-----------------+---------+-------+----------------+-------------------------------+--------------------+
|Codigo_IATA|       Aeropuerto|Id_Estado|Aniomes|Tipo_combustible|Litros_combustible_suministrado|Servicios_realizados|
+-----------+-----------------+---------+-------+----------------+-------------------------------+--------------------+
|        ACA|         Acapulco|       12| 201501|       Turbosina|                        1661674|                 757|
|        AGU|   Aguascalientes|        1| 201501|       Turbosina|                        1557391|                 502|
|        BJX|         El Bajio|       11| 201501|       Turbosina|                        3975683|                1124|
|        CEN|   Ciudad Obregon|       26| 201501|       Turbosina|                         641425|                 240|
|        CJS|    Ciudad Juarez|        8| 201501|       Turbosina|                        2032172|                 502|
|        CME|Ciudad Del Carmen|        4

In [264]:
#Dataframe "Estados"
print(dfe.show(32))
print(dfe.count())

+-----------+-----+--------------------+
|abreviatura|clave|              estado|
+-----------+-----+--------------------+
|       Ags.|    1|      Aguascalientes|
|       Dgo.|   10|             Durango|
|       Gto.|   11|          Guanajuato|
|       Gro.|   12|            Guerrero|
|       Hgo.|   13|             Hidalgo|
|       Jal.|   14|             Jalisco|
|       Mex.|   15|              México|
|      Mich.|   16| Michoacán de Ocampo|
|       Mor.|   17|             Morelos|
|       Nay.|   18|             Nayarit|
|         NL|   19|          Nuevo León|
|         BC|    2|     Baja California|
|       Oax.|   20|              Oaxaca|
|       Pue.|   21|              Puebla|
|       Qro.|   22|           Querétaro|
|     Q. Roo|   23|        Quintana Roo|
|        SLP|   24|     San Luis Potosí|
|       Sin.|   25|             Sinaloa|
|       Son.|   26|              Sonora|
|       Tab.|   27|             Tabasco|
|     Tamps.|   28|          Tamaulipas|
|      Tlax.|   

In [246]:
#Creacion de tablas temporales
dfp.createOrReplaceTempView("servicios")
dfe.createOrReplaceTempView("estados")

spark.sql("SHOW TABLES").show()

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
|         |  estados|       true|
|         |servicios|       true|
+---------+---------+-----------+



In [257]:
#Se muestra que el DataFrame "Servicios" tiene un error en el "id_estado", ya que hay en el "codigo_iata"
#se marca que existen dos aeropuertos "Mexico", uno con "id_estado= 9" y otro con "id_estado= DF"
sql="""
SELECT servicios.aeropuerto,servicios.codigo_iata, servicios.id_estado
FROM estados
JOIN servicios
GROUP BY servicios.aeropuerto,servicios.codigo_iata, servicios.id_estado
ORDER BY 1
"""
spark.sql(sql).show(50)
print(spark.sql(sql).count())

+-----------------+-----------+---------+
|       aeropuerto|codigo_iata|id_estado|
+-----------------+-----------+---------+
|         Acapulco|        ACA|       12|
|   Aguascalientes|        AGU|        1|
|         Campeche|        CPE|        4|
|           Cancun|        CUN|       23|
|         Chetumal|        CTM|       23|
|        Chihuahua|        CUU|        8|
|Ciudad Del Carmen|        CME|        4|
|    Ciudad Juarez|        CJS|        8|
|   Ciudad Obregon|        CEN|       26|
|  Ciudad Victoria|        CVM|       28|
|           Colima|        CLQ|        6|
|          Cozumel|        CZM|       23|
|       Cuernavaca|        CVJ|       17|
|         Culiacan|        CUL|       25|
|          Durango|        DGO|       10|
|         El Bajio|        BJX|       11|
|   Felipe Angeles|        NLU|       15|
|      Guadalajara|        GDL|       14|
|          Guaymas|        GYM|       26|
|       Hermosillo|        HMO|       26|
|         Huatulco|        HUX|   

In [248]:
#Comprobacion de que existen dos aeropuertos "Mexico", uno con "id_estado= 9" y otro con "id_estado= DF"
sql1=""" SELECT servicios.id_estado, servicios.codigo_iata
FROM servicios
JOIN estados
ON servicios.id_estado=9
"""
spark.sql(sql1).show()
print(spark.sql(sql1).count())

sql=""" SELECT servicios.id_estado, servicios.codigo_iata
FROM servicios
JOIN estados
ON servicios.id_estado='DF'
"""
spark.sql(sql).show()
print(spark.sql(sql).count())

#Debido a que SparkSQL no actualiza el campo de 'DF' a '9', los procesos se realizaron de esta manera

+---------+-----------+
|id_estado|codigo_iata|
+---------+-----------+
|        9|        MEX|
|        9|        MEX|
|        9|        MEX|
|        9|        MEX|
|        9|        MEX|
|        9|        MEX|
|        9|        MEX|
|        9|        MEX|
|        9|        MEX|
|        9|        MEX|
|        9|        MEX|
|        9|        MEX|
|        9|        MEX|
|        9|        MEX|
|        9|        MEX|
|        9|        MEX|
|        9|        MEX|
|        9|        MEX|
|        9|        MEX|
|        9|        MEX|
+---------+-----------+
only showing top 20 rows

5376
+---------+-----------+
|id_estado|codigo_iata|
+---------+-----------+
|       DF|        MEX|
|       DF|        MEX|
|       DF|        MEX|
|       DF|        MEX|
|       DF|        MEX|
|       DF|        MEX|
|       DF|        MEX|
|       DF|        MEX|
|       DF|        MEX|
|       DF|        MEX|
|       DF|        MEX|
|       DF|        MEX|
|       DF|        MEX|
|       D

In [250]:
#Número total de servicios realizados por estado
sql = """
SELECT estados.clave, estados.estado, SUM(servicios.servicios_realizados) no_servicios
FROM servicios
JOIN estados
ON servicios.id_estado= estados.clave
GROUP BY estados.clave, estados.estado
ORDER BY estados.estado
"""
spark.sql(sql).show(31)
print(spark.sql(sql).count())

+-----+--------------------+------------+
|clave|              estado|no_servicios|
+-----+--------------------+------------+
|    1|      Aguascalientes|       53805|
|    2|     Baja California|      305516|
|    3| Baja California Sur|      272027|
|    4|            Campeche|       63264|
|    7|             Chiapas|       93666|
|    8|           Chihuahua|      177512|
|    9|    Ciudad de México|     1387227|
|    5|Coahuila de Zaragoza|       54690|
|    6|              Colima|       38050|
|   10|             Durango|       57462|
|   11|          Guanajuato|      118135|
|   12|            Guerrero|      130845|
|   13|             Hidalgo|        9835|
|   14|             Jalisco|      687600|
|   16| Michoacán de Ocampo|       64912|
|   17|             Morelos|       65561|
|   15|              México|      230196|
|   18|             Nayarit|       21554|
|   19|          Nuevo León|      398649|
|   20|              Oaxaca|      116478|
|   21|              Puebla|      

In [258]:
#Número total de servicios realizados por aeropuerto, se muestra el estado
sql = """
SELECT estados.clave, estados.estado, servicios.aeropuerto,
SUM(servicios.servicios_realizados) no_servicios
FROM servicios
JOIN estados
ON servicios.id_estado= estados.clave
GROUP BY estados.clave, estados.estado,servicios.aeropuerto
ORDER BY estados.estado
"""
spark.sql(sql).show(50)
print(spark.sql(sql).count())

+-----+--------------------+-----------------+------------+
|clave|              estado|       aeropuerto|no_servicios|
+-----+--------------------+-----------------+------------+
|    1|      Aguascalientes|   Aguascalientes|       53805|
|    2|     Baja California|          Tijuana|      263616|
|    2|     Baja California|         Mexicali|       41900|
|    3| Baja California Sur|           La Paz|       63127|
|    3| Baja California Sur|           Loreto|       13972|
|    3| Baja California Sur|San Jose Del Cabo|      194928|
|    4|            Campeche|         Campeche|       18146|
|    4|            Campeche|Ciudad Del Carmen|       45118|
|    7|             Chiapas|         Palenque|        1759|
|    7|             Chiapas|        Tapachula|       36237|
|    7|             Chiapas| Tuxtla Gutierrez|       55670|
|    8|           Chihuahua|        Chihuahua|      114554|
|    8|           Chihuahua|    Ciudad Juarez|       62958|
|    9|    Ciudad de México|           M

In [259]:
#Maximo, minimo, promedio y total de litros de combustible por estado
sql = """
SELECT estados.clave, estados.estado,
AVG(servicios.litros_combustible_suministrado) litros_promedio,
SUM(servicios.litros_combustible_suministrado) total_litros,
MAX(servicios.litros_combustible_suministrado) max_litros,
MIN(servicios.litros_combustible_suministrado) min_litros
FROM servicios
JOIN estados
ON servicios.id_estado= estados.clave
GROUP BY estados.clave, estados.estado
ORDER BY estados.estado
"""
spark.sql(sql).show(31)
print(spark.sql(sql).count())

+-----+--------------------+--------------------+------------+----------+----------+
|clave|              estado|     litros_promedio|total_litros|max_litros|min_litros|
+-----+--------------------+--------------------+------------+----------+----------+
|    1|      Aguascalientes|   850269.7239583334|   163251787|   2231356|      4405|
|    2|     Baja California|   5293568.791666667|  2032730416|  30156947|      3552|
|    3| Baja California Sur|  2261269.6319444445|  1302491308|  18860897|         0|
|    4|            Campeche|      541523.9765625|   207945207|   2693424|         0|
|    7|             Chiapas|   556496.3489583334|   320541897|   2924236|         0|
|    8|           Chihuahua|  1468393.5963541667|   563863141|   4298356|         0|
|    9|    Ciudad de México| 6.793585186309524E7| 11413223113| 175193398|         0|
|    5|Coahuila de Zaragoza|   608295.3072916666|   116792699|   1774353|      6250|
|    6|              Colima|      219176.1953125|    84163659|   

In [261]:
#Tabla pivote mostrar total de litros de combustible por estado y tipo (Turbosina,Gasavion)
sql=""" SELECT estados.clave, estados.estado, servicios.tipo_combustible,
SUM(servicios.litros_combustible_suministrado) total_litros
FROM servicios
JOIN estados
ON servicios.id_estado =estados.clave
GROUP BY estados.clave, estados.estado,servicios.tipo_combustible
ORDER BY estados.estado
"""

spark.sql(sql).show(50)
print(spark.sql(sql).count())

+-----+--------------------+----------------+------------+
|clave|              estado|tipo_combustible|total_litros|
+-----+--------------------+----------------+------------+
|    1|      Aguascalientes|        Gasavion|     1015666|
|    1|      Aguascalientes|       Turbosina|   162236121|
|    2|     Baja California|       Turbosina|  2025350381|
|    2|     Baja California|        Gasavion|     7380035|
|    3| Baja California Sur|        Gasavion|     4779440|
|    3| Baja California Sur|       Turbosina|  1297711868|
|    4|            Campeche|       Turbosina|   206942000|
|    4|            Campeche|        Gasavion|     1003207|
|    7|             Chiapas|       Turbosina|   305998954|
|    7|             Chiapas|        Gasavion|    14542943|
|    8|           Chihuahua|        Gasavion|    13461051|
|    8|           Chihuahua|       Turbosina|   550402090|
|    9|    Ciudad de México|        Gasavion|     1580564|
|    9|    Ciudad de México|       Turbosina| 1141164254

In [262]:
#Mostrar 5 aeropuertos con mayor numero de servicios
sql = """
SELECT estados.clave, estados.estado, servicios.aeropuerto,
SUM(servicios.servicios_realizados) no_servicios
FROM servicios
JOIN estados
ON servicios.id_estado= estados.clave
GROUP BY estados.clave, estados.estado, servicios.aeropuerto
ORDER BY no_servicios DESC
LIMIT 5
"""
spark.sql(sql).show(63)
print(spark.sql(sql).count())

+-----+----------------+-----------+------------+
|clave|          estado| aeropuerto|no_servicios|
+-----+----------------+-----------+------------+
|    9|Ciudad de México|     Mexico|     1387227|
|   23|    Quintana Roo|     Cancun|      662959|
|   14|         Jalisco|Guadalajara|      527900|
|   19|      Nuevo León|  Monterrey|      398649|
|    2| Baja California|    Tijuana|      263616|
+-----+----------------+-----------+------------+

5


In [263]:
#Mostrar 10 estados con menos consumo de Turbosina
sql=""" SELECT estados.clave, estados.estado, servicios.tipo_combustible,
SUM(servicios.litros_combustible_suministrado) total_litros
FROM servicios
JOIN estados
ON servicios.id_estado =estados.clave
WHERE servicios.tipo_combustible='Turbosina'
GROUP BY estados.clave, estados.estado,servicios.tipo_combustible
ORDER BY total_litros
LIMIT 10
"""
spark.sql(sql).show()
print(spark.sql(sql).count())

+-----+--------------------+----------------+------------+
|clave|              estado|tipo_combustible|total_litros|
+-----+--------------------+----------------+------------+
|   13|             Hidalgo|       Turbosina|     3805464|
|   17|             Morelos|       Turbosina|    10133727|
|   18|             Nayarit|       Turbosina|    36555747|
|    6|              Colima|       Turbosina|    73397599|
|   10|             Durango|       Turbosina|    83155542|
|   32|           Zacatecas|       Turbosina|    86152573|
|    5|Coahuila de Zaragoza|       Turbosina|   114372837|
|   21|              Puebla|       Turbosina|   133657968|
|   24|     San Luis Potosí|       Turbosina|   143450981|
|    1|      Aguascalientes|       Turbosina|   162236121|
+-----+--------------------+----------------+------------+

10
